# EJERCICIOS

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_date, year, monotonically_increasing_id,  avg, min, coalesce, col, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

spark = SparkSession.builder.master("local[*]").appName("pyspark_rdd").getOrCreate()
#spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

25/02/06 15:48:15 WARN Utils: Your hostname, iadb-12 resolves to a loopback address: 127.0.1.1; using 172.20.104.44 instead (on interface wlp44s0f0)
25/02/06 15:48:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/06 15:48:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/02/06 15:48:28 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## EJERCICIO 0
En un documento word haz una lista de las diferentes operaciones con una breve descripción de lo que hace y un ejemplo de como se utiliza

## EJERCICIO 1
Realiza las siguientes operaciones:
* Importa el csv de "data/WorldCupPlayers.csv" (que deduzca el esquema)
* Visualiza los datos
* ¿Que tipo de datos contiene cada variable?
* ¿Cuantos registros hay?
* Obtén los principales estadísticos de Position
* Selecciona y muestra los "Team initials" diferentes que hay ¿Cuántos hay?
* ¿Cuantos partidos con el ID de 1096 ha habido?
* Muestra los datos donde la posicion haya sido C y el evento sea G40
* Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20

In [7]:
dfPlayers = (
    spark.read.option("sep", ",")
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("Datos/WorldCupPlayers.csv")
)

In [6]:
dfPlayers.show(1)

+-------+-------+-------------+-------------------+-------+------------+-----------+--------+-----+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|Player Name|Position|Event|
+-------+-------+-------------+-------------------+-------+------------+-----------+--------+-----+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|Alex THEPOT|      GK| NULL|
+-------+-------+-------------+-------------------+-------+------------+-----------+--------+-----+
only showing top 1 row



In [8]:
dfPlayers.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [9]:
dfPlayers.count()

37784

In [12]:
dfPlayers.describe("position").show()

+-------+--------+
|summary|position|
+-------+--------+
|  count|    4143|
|   mean|    NULL|
| stddev|    NULL|
|    min|       C|
|    max|     GKC|
+-------+--------+



In [13]:
dfPlayers.select("Team Initials").distinct().show()

+-------------+
|Team Initials|
+-------------+
|          POL|
|          JAM|
|          BRA|
|          CUB|
|          FRA|
|          ALG|
|          BOL|
|          RSA|
|          ITA|
|          UKR|
|          CMR|
|          SCG|
|          GHA|
|          SEN|
|          TOG|
|          TRI|
|          TCH|
|          AUS|
|          MEX|
|          PAR|
+-------------+
only showing top 20 rows



In [16]:
dfPlayers.filter(dfPlayers.MatchID == 1096).count()

33

In [17]:
dfPlayers.filter((dfPlayers.Event == "G40") & (dfPlayers.Position == "C")).show()

+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
|RoundID|MatchID|Team Initials|Coach Name|Line-up|Shirt Number|Player Name|Position|Event|
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+



In [18]:
dfPlayers.createOrReplaceTempView("players_table")
spark.sql("SELECT * FROM players_table WHERE MatchID >= 20").show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

## EJERCICIO 2

A partir del archivo nombres.json, crea un DataFrame y realiza las siguientes operaciones:

1. Crea una nueva columna (columna Mayor30) que indique si la persona es mayor de 30 años.
2. Crea una nueva columna (columna FaltanJubilacion) que calcule cuantos años le faltan para jubilarse (supongamos que se jubila a los 67 años)
3. Crea una nueva columna (columna Apellidos) que contenga XYZ (puedes utilizar la función lit)
4. Elimina las columna Mayor30 y Apellidos.
5. Crea una nueva columna (columna AnyoNac) con el año de nacimiento de cada persona (puedes utilizar la función current_date).
6. Añade un id incremental para cada fila (campo Id) y haz que al hacer un show se vea en primer lugar (puedes utilizar la función monotonically_increasing_id) seguidos del Nombre, Edad, AnyoNac, FaltaJubilacion y Ciudad

Al realizar los seis pasos, el resultado del DataFrame será similar a :
``````
+---+-------+----+-------+----------------+--------+
| Id|Nombre |Edad|AnyoNac|FaltanJubilacion|  Ciudad|
+---+-------+----+-------+----------------+--------+
|  0|  Aitor|  45|   1977|              22|   Elche|
|  1| Marina|  14|   2008|              53|Alicante|
|  2|  Laura|  19|   2003|              48|   Elche|
|  3|  Sonia|  45|   1977|              22|    Aspe|
|  4|  Pedro|null|   null|            null|   Elche|
+---+-------+----+-------+----------------+--------+
``````

In [24]:
dfNombre = spark.read.json("Datos/nombres.json")
dfNombre.show(1)

+------+----+------+
|Ciudad|Edad|Nombre|
+------+----+------+
| Elche|  45| Aitor|
+------+----+------+
only showing top 1 row



In [25]:
dfNombre = dfNombre.withColumn("mayor30", when(col("Edad") > 30, "Y").otherwise("N"))
dfNombre.show(1)

+------+----+------+-------+
|Ciudad|Edad|Nombre|mayor30|
+------+----+------+-------+
| Elche|  45| Aitor|      Y|
+------+----+------+-------+
only showing top 1 row



In [26]:
dfNombre = dfNombre.withColumn("FaltaJubilacion", 67 - col("Edad"))
dfNombre.show(1)

+------+----+------+-------+---------------+
|Ciudad|Edad|Nombre|mayor30|FaltaJubilacion|
+------+----+------+-------+---------------+
| Elche|  45| Aitor|      Y|             22|
+------+----+------+-------+---------------+
only showing top 1 row



In [27]:
dfNombre = dfNombre.withColumn("Apellidos", lit("XYZ"))
dfNombre.show(1)

+------+----+------+-------+---------------+---------+
|Ciudad|Edad|Nombre|mayor30|FaltaJubilacion|Apellidos|
+------+----+------+-------+---------------+---------+
| Elche|  45| Aitor|      Y|             22|      XYZ|
+------+----+------+-------+---------------+---------+
only showing top 1 row



In [28]:
dfNombre = dfNombre.drop("Apellidos", "mayor30")
dfNombre.show(1)

+------+----+------+---------------+
|Ciudad|Edad|Nombre|FaltaJubilacion|
+------+----+------+---------------+
| Elche|  45| Aitor|             22|
+------+----+------+---------------+
only showing top 1 row



In [29]:
dfNombre = dfNombre.withColumn("AnyoNac", year(current_date()) - col("Edad"))
dfNombre.show(1)

+------+----+------+---------------+-------+
|Ciudad|Edad|Nombre|FaltaJubilacion|AnyoNac|
+------+----+------+---------------+-------+
| Elche|  45| Aitor|             22|   1980|
+------+----+------+---------------+-------+
only showing top 1 row



In [30]:
dfNombre = dfNombre.withColumn("Id", monotonically_increasing_id())
dfNombre = dfNombre.select("Id", "Nombre", "Edad", "AnyoNac", "FaltaJubilacion", "Ciudad")
dfNombre.show()

+---+------+----+-------+---------------+--------+
| Id|Nombre|Edad|AnyoNac|FaltaJubilacion|  Ciudad|
+---+------+----+-------+---------------+--------+
|  0| Aitor|  45|   1980|             22|   Elche|
|  1|Marina|  14|   2011|             53|Alicante|
|  2| Laura|  19|   2006|             48|   Elche|
|  3| Sonia|  45|   1980|             22|    Aspe|
|  4| Pedro|NULL|   NULL|           NULL|   Elche|
+---+------+----+-------+---------------+--------+



## EJERCICIO 3

A partir del archivo VentasNulos.csv:

1. Elimina las filas que tengan al menos 4 nulos.

2. Con las filas restantes, sustituye:

    * Los nombres nulos por Empleado
    * Las ventas nulas por la media de las ventas de los compañeros (redondeado a entero).
    ``````
        media = df.groupBy().avg('Ventas')
    ``````
    * Los euros nulos por el valor del compañero que menos € ha ganado. (tras agrupar, puedes usar la función min)
    * La ciudad nula por C.V. y el identificador nulo por XYZ

In [10]:
dfNull = (
    spark.read.option("sep", ",")
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("Datos/VentasNulos.csv")
)
dfNull.show()

+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| NULL|       NULL|         NULL|
|  Juan|  NULL| NULL|       NULL|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  NULL|  300| Torrellano|         NULL|
|Marina|     3|  350|       Aspe|          V55|
|  NULL|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
|  NULL|  NULL| NULL|       NULL|         NULL|
| Jorge|     8| NULL|       NULL|          T19|
+------+------+-----+-----------+-------------+



In [8]:
dfNull = dfNull.filter(sum(col(c).isNull().cast("int") for c in dfNull.columns) < 4)
dfNull.show()

+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| NULL|       NULL|         NULL|
|  Juan|  NULL| NULL|       NULL|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  NULL|  300| Torrellano|         NULL|
|Marina|     3|  350|       Aspe|          V55|
|  NULL|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
| Jorge|     8| NULL|       NULL|          T19|
+------+------+-----+-----------+-------------+



In [11]:
dfNull = dfNull.na.fill("Empleado",subset="Nombre")
dfNull.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8| NULL|       NULL|         NULL|
|    Juan|  NULL| NULL|       NULL|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|  NULL|  300| Torrellano|         NULL|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|  NULL| NULL|       NULL|         NULL|
|   Jorge|     8| NULL|       NULL|          T19|
+--------+------+-----+-----------+-------------+



In [13]:
media = dfNull.select(avg("Ventas")).collect()[0][0]
dfNull = dfNull.na.fill({"Ventas": media})
dfNull.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8| NULL|       NULL|         NULL|
|    Juan|     6| NULL|       NULL|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|         NULL|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6| NULL|       NULL|         NULL|
|   Jorge|     8| NULL|       NULL|          T19|
+--------+------+-----+-----------+-------------+



In [14]:
min_euros = dfNull.select(min("Euros")).collect()[0][0]
dfNull = dfNull.na.fill({"Euros": min_euros})
dfNull.show()


+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8|   30|       NULL|         NULL|
|    Juan|     6|   30|       NULL|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|         NULL|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6|   30|       NULL|         NULL|
|   Jorge|     8|   30|       NULL|          T19|
+--------+------+-----+-----------+-------------+



In [15]:
dfNull = dfNull.na.fill({"Ciudad":"C.V."})
dfNull = dfNull.na.fill({"Identificador": "XYZ"})
dfNull.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8|   30|       C.V.|          XYZ|
|    Juan|     6|   30|       C.V.|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|          XYZ|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6|   30|       C.V.|          XYZ|
|   Jorge|     8|   30|       C.V.|          T19|
+--------+------+-----+-----------+-------------+



## EJERCICIO 4

 A partir del archivo movies.tsv, crea una esquema de forma declarativa con los campos:

* interprete de tipo string
* pelicula de tipo string
* anyo de tipo int

Cada fila del fichero implica que el actor/actriz ha trabajado en dicha película en el año indicado.
1. Una vez creado el esquema, carga los datos en un DataFrame.

A continuación, mediante el DataFrame API:

2. Muestra las películas en las que ha trabajado Murphy, Eddie (I).
3. Muestra los intérpretes que aparecen tanto en Superman como en Superman II.

## EJERCICIO 5
Realiza las siguientes operaciones:
* Carga el dataset de “data/stocks_price_final.csv”, con el esquema correcto de datos (tienes que crear tu el schema").
* Renombra la variable market.cap a market
* Elimina la variable market
* Muestra las filas donde el valor de "open" es nulo.
* Elimina las filas donde el valor de "open" es nulo.
* Para comprobar el punto anterior vuelve a mostrar las filas donde el valor de "open" es nulo.
